In [1]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [2]:
from utils.config import Config
from datasets.CellData import CellDataset, CellDataCollate, CellDataCustomBatch, CellDataSampler
from models.architectures import KPFCNN
from torch.utils.data import DataLoader
from utils.trainer import ModelTrainer
import numpy as np
import torch

In [3]:
class CellDataConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'Fluo-C3DH-A549'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = 'cloud_segmentation'

    # Number of CPU threads for the input pipeline
    input_threads = 0

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 2.0 #2.0

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.08

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 2.5 #2.5

    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.2

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'gaussian' #linear

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 1

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-4
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 10

    # Number of steps per epochs
    epoch_steps = 100

    # Number of validation examples per epoch
    validation_size = 50

    # Number of epoch between each checkpoint
    checkpoint_gap = 10

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'class'
    class_w = [0.02, 0.98]

    # Do we need to save convergence
    saving = True
    saving_path = None

In [4]:
config = CellDataConfig()
train_dataset = CellDataset(config, set = 'training')
validation_dataset = CellDataset(config, set = 'validation')

training_sampler = CellDataSampler(train_dataset)
validation_sampler = CellDataSampler(validation_dataset)


Preparing KDTree for cloud t000, subsampled at 0.080
14.7 MB loaded in 5.9s

Preparing KDTree for cloud t002, subsampled at 0.080
14.7 MB loaded in 10.7s

Preparing KDTree for cloud t006, subsampled at 0.080
14.7 MB loaded in 7.9s

Preparing KDTree for cloud t008, subsampled at 0.080
14.7 MB loaded in 7.5s

Preparing KDTree for cloud t012, subsampled at 0.080
14.7 MB loaded in 6.2s

Preparing KDTree for cloud t013, subsampled at 0.080
14.7 MB loaded in 6.4s

Preparing KDTree for cloud t014, subsampled at 0.080
14.7 MB loaded in 6.5s

Preparing KDTree for cloud t019, subsampled at 0.080
14.7 MB loaded in 6.4s

Preparing KDTree for cloud t022, subsampled at 0.080
14.7 MB loaded in 5.5s

Preparing KDTree for cloud t025, subsampled at 0.080
14.7 MB loaded in 5.2s

Preparing KDTree for cloud t026, subsampled at 0.080
14.7 MB loaded in 4.2s

Preparing KDTree for cloud t027, subsampled at 0.080
14.7 MB loaded in 5.3s

Preparing KDTree for cloud t028, subsampled at 0.080
14.7 MB loaded in 3.3

In [5]:
training_loader = DataLoader(train_dataset,
                            collate_fn=CellDataCollate,
                            sampler=training_sampler,)

validation_loader = DataLoader(validation_dataset,
                              collate_fn=CellDataCollate,
                              sampler=training_sampler,)

training_sampler.calibration(training_loader)
validation_sampler.calibration(validation_loader)

net = KPFCNN(config, train_dataset.label_values, train_dataset.ignored_labels)
trainer = ModelTrainer(net, config)


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s



In [ ]:
trainer.train(net, training_loader, validation_loader, config)

e000-i0000 => L=19.313 acc= 55% / t(ms):  51.8 3473.9 267.8)
e000-i0002 => L=19.400 acc= 70% / t(ms):  43.2 230.9 262.4)
e000-i0004 => L=19.115 acc= 74% / t(ms):  42.9 227.0 262.7)
e000-i0006 => L=19.236 acc= 65% / t(ms):  42.6 225.2 261.0)
e000-i0008 => L=18.975 acc= 67% / t(ms):  46.8 222.3 259.1)
e000-i0011 => L=18.970 acc= 70% / t(ms):  44.2 219.5 257.9)
e000-i0013 => L=18.820 acc= 80% / t(ms):  43.9 218.6 256.5)
e000-i0015 => L=19.050 acc= 71% / t(ms):  43.8 217.1 258.6)
e000-i0017 => L=18.746 acc= 76% / t(ms):  43.1 217.7 256.7)
e000-i0019 => L=18.821 acc= 77% / t(ms):  42.9 217.4 258.5)
e000-i0021 => L=18.456 acc= 77% / t(ms):  42.6 217.2 258.0)
e000-i0023 => L=18.626 acc= 78% / t(ms):  42.6 216.2 256.6)
e000-i0025 => L=18.545 acc= 76% / t(ms):  42.5 216.9 256.3)
e000-i0027 => L=18.602 acc= 78% / t(ms):  42.6 215.7 254.9)
e000-i0029 => L=18.118 acc= 83% / t(ms):  42.2 215.6 256.8)
e000-i0031 => L=18.379 acc= 84% / t(ms):  41.7 217.4 256.2)
e000-i0033 => L=17.738 acc= 87% / t(ms)

e002-i0026 => L=13.004 acc= 89% / t(ms):  55.2 294.5 320.2)
e002-i0028 => L=12.587 acc= 86% / t(ms):  56.1 301.7 326.6)
e002-i0030 => L=12.682 acc= 93% / t(ms):  57.8 302.8 333.1)
e002-i0032 => L=12.540 acc= 88% / t(ms):  58.9 306.2 334.2)
e002-i0034 => L=13.066 acc= 82% / t(ms):  59.7 305.9 336.8)
e002-i0036 => L=12.547 acc= 90% / t(ms):  59.4 308.9 337.4)
e002-i0038 => L=12.654 acc= 95% / t(ms):  57.0 301.9 326.3)
e002-i0040 => L=12.359 acc= 89% / t(ms):  55.2 296.2 316.7)
e002-i0042 => L=12.439 acc= 88% / t(ms):  53.6 289.9 310.4)
e002-i0044 => L=12.493 acc= 88% / t(ms):  52.5 284.3 303.6)
e002-i0046 => L=12.408 acc= 90% / t(ms):  52.3 280.6 299.7)
e002-i0048 => L=12.598 acc= 87% / t(ms):  51.4 275.6 294.7)
e002-i0050 => L=12.463 acc= 89% / t(ms):  50.0 272.2 292.4)
e002-i0052 => L=12.435 acc= 86% / t(ms):  48.7 269.8 289.0)
e002-i0054 => L=12.746 acc= 87% / t(ms):  47.9 267.9 287.1)
e002-i0056 => L=12.749 acc= 86% / t(ms):  47.7 265.5 284.6)
e002-i0058 => L=12.442 acc= 89% / t(ms):

e004-i0044 => L=12.499 acc= 84% / t(ms):  59.5 318.6 332.8)
e004-i0046 => L=12.571 acc= 95% / t(ms):  57.6 305.8 321.4)
e004-i0048 => L=12.557 acc= 85% / t(ms):  54.3 298.6 312.8)
e004-i0050 => L=12.925 acc= 89% / t(ms):  52.3 292.4 305.3)
e004-i0052 => L=12.934 acc= 86% / t(ms):  51.2 285.6 303.2)
e004-i0054 => L=12.583 acc= 84% / t(ms):  50.0 279.9 298.1)
e004-i0056 => L=12.532 acc= 93% / t(ms):  48.9 277.4 295.5)
e004-i0058 => L=13.284 acc= 79% / t(ms):  48.0 275.7 292.0)
e004-i0060 => L=12.474 acc= 85% / t(ms):  47.4 273.1 288.5)
e004-i0062 => L=12.644 acc= 83% / t(ms):  48.6 269.6 287.6)
e004-i0064 => L=12.547 acc= 90% / t(ms):  47.8 266.0 287.6)
e004-i0066 => L=12.498 acc= 91% / t(ms):  48.1 263.9 287.2)
e004-i0068 => L=12.452 acc= 88% / t(ms):  47.9 264.0 288.3)
e004-i0070 => L=12.478 acc= 88% / t(ms):  48.7 269.7 294.2)
e004-i0072 => L=12.442 acc= 94% / t(ms):  51.5 282.9 300.9)
e004-i0074 => L=12.874 acc= 86% / t(ms):  54.0 291.1 312.4)
e004-i0076 => L=12.404 acc= 93% / t(ms):

e006-i0058 => L=12.229 acc= 86% / t(ms):  50.4 279.5 297.1)
e006-i0060 => L=13.454 acc= 85% / t(ms):  49.4 276.5 293.4)
e006-i0061 => L=12.550 acc= 87% / t(ms): 178.0 272.1 290.4)
e006-i0063 => L=12.221 acc= 84% / t(ms): 152.3 266.5 285.3)
e006-i0065 => L=12.179 acc= 84% / t(ms): 131.2 262.0 282.1)
e006-i0067 => L=12.131 acc= 94% / t(ms): 114.6 259.2 280.6)
e006-i0069 => L=12.195 acc= 91% / t(ms): 101.7 257.6 279.4)
e006-i0071 => L=12.095 acc= 91% / t(ms):  99.5 259.6 282.1)
e006-i0073 => L=12.120 acc= 86% / t(ms):  91.5 269.7 294.6)
e006-i0075 => L=12.361 acc= 87% / t(ms):  85.4 284.3 306.1)
e006-i0077 => L=12.185 acc= 93% / t(ms):  80.6 293.3 316.4)
e006-i0079 => L=12.277 acc= 87% / t(ms):  77.4 302.6 325.1)
e006-i0081 => L=12.544 acc= 86% / t(ms):  75.1 308.5 330.9)
e006-i0083 => L=12.483 acc= 89% / t(ms):  72.3 315.7 337.3)
e006-i0085 => L=12.285 acc= 88% / t(ms):  71.1 320.7 338.0)
e006-i0087 => L=12.527 acc= 87% / t(ms):  70.1 322.0 342.2)
e006-i0089 => L=12.334 acc= 93% / t(ms):

e008-i0070 => L=12.116 acc= 88% / t(ms):  49.1 270.1 291.0)
e008-i0072 => L=12.154 acc= 88% / t(ms):  48.4 267.3 287.2)
e008-i0074 => L=11.914 acc= 88% / t(ms):  47.4 263.6 283.8)
e008-i0076 => L=12.121 acc= 88% / t(ms):  47.9 261.9 282.5)
e008-i0078 => L=11.918 acc= 94% / t(ms):  47.4 262.1 281.3)
e008-i0080 => L=12.215 acc= 89% / t(ms):  47.2 262.6 282.2)
e008-i0082 => L=11.984 acc= 87% / t(ms):  47.3 266.2 285.5)
e008-i0084 => L=12.336 acc= 83% / t(ms):  49.2 272.9 293.0)
e008-i0086 => L=12.154 acc= 88% / t(ms):  52.3 284.6 301.7)
e008-i0088 => L=12.042 acc= 91% / t(ms):  54.0 292.3 311.3)
e008-i0090 => L=12.080 acc= 91% / t(ms):  56.5 298.9 318.8)
e008-i0092 => L=11.945 acc= 93% / t(ms):  57.2 302.8 324.6)
e008-i0094 => L=12.106 acc= 87% / t(ms):  58.9 306.5 326.8)
e008-i0096 => L=11.960 acc= 92% / t(ms):  59.0 311.8 330.9)
e008-i0098 => L=12.105 acc= 87% / t(ms):  59.2 314.0 332.3)
Validation : 10.0% (timings : 16.89 32.35)
Validation : 20.0% (timings : 30.42 57.82)
Validation : 3

...

e345-i0000 => L=11.613 acc= 74% / t(ms): 20023.8 326.2 343.4)
e345-i0002 => L=11.233 acc= 86% / t(ms):  70.2 391.2 371.8)
e345-i0004 => L=11.108 acc= 83% / t(ms):  67.9 377.2 363.4)
e345-i0006 => L=11.895 acc= 87% / t(ms):  66.2 367.0 360.4)
e345-i0008 => L=11.196 acc= 88% / t(ms):  63.7 361.3 354.7)
e345-i0010 => L=11.120 acc= 94% / t(ms):  62.5 353.1 350.6)
e345-i0012 => L=11.174 acc= 86% / t(ms):  60.4 338.3 336.6)
e345-i0014 => L=10.965 acc=100% / t(ms):  57.4 323.4 322.8)
e345-i0016 => L=11.234 acc= 90% / t(ms):  54.4 311.5 313.1)
e345-i0018 => L=10.998 acc= 95% / t(ms):  52.4 299.9 307.8)
e345-i0020 => L=11.160 acc= 85% / t(ms):  50.5 290.4 300.4)
e345-i0022 => L=11.185 acc= 87% / t(ms):  51.7 288.6 294.8)
e345-i0024 => L=11.093 acc= 94% / t(ms):  50.3 280.1 289.7)
e345-i0026 => L=11.143 acc= 81% / t(ms):  49.1 278.8 285.5)
e345-i0028 => L=11.002 acc= 98% / t(ms):  47.6 276.1 281.5)
e345-i0030 => L=11.744 acc= 94% / t(ms):  46.7 278.0 278.5)
e345-i0032 => L=11.892 acc= 82% / t(ms

e347-i0020 => L=10.917 acc=100% / t(ms):  62.5 268.3 293.0)
e347-i0022 => L=11.134 acc= 96% / t(ms):  60.2 276.3 294.6)
e347-i0024 => L=11.217 acc= 87% / t(ms):  57.6 278.7 297.4)
e347-i0026 => L=11.022 acc=100% / t(ms):  55.8 279.3 299.4)
e347-i0028 => L=10.967 acc=100% / t(ms):  54.8 283.5 303.2)
e347-i0030 => L=11.020 acc= 94% / t(ms):  54.8 287.8 301.1)
e347-i0032 => L=11.056 acc= 94% / t(ms):  53.7 283.9 301.0)
e347-i0034 => L=10.980 acc= 91% / t(ms):  52.6 283.1 302.7)
e347-i0036 => L=11.069 acc=100% / t(ms):  52.4 285.5 301.7)
e347-i0038 => L=11.141 acc= 94% / t(ms):  51.8 286.3 301.3)
e347-i0040 => L=11.047 acc= 93% / t(ms):  52.4 294.8 299.5)
e347-i0042 => L=10.997 acc= 98% / t(ms):  51.7 297.3 295.1)
e347-i0044 => L=11.137 acc= 93% / t(ms):  49.1 285.1 287.4)
e347-i0046 => L=10.936 acc= 89% / t(ms):  47.1 272.3 280.1)
e347-i0048 => L=11.159 acc= 99% / t(ms):  46.2 262.6 274.6)
e347-i0050 => L=10.945 acc= 93% / t(ms):  44.4 254.4 269.9)
e347-i0052 => L=11.513 acc= 84% / t(ms):

e349-i0045 => L=11.058 acc= 84% / t(ms):  52.0 282.4 292.6)
e349-i0047 => L=11.073 acc= 87% / t(ms):  51.8 288.1 293.1)
e349-i0049 => L=11.199 acc= 82% / t(ms):  52.3 291.4 301.7)
e349-i0051 => L=11.301 acc= 93% / t(ms):  55.0 299.8 306.9)
e349-i0053 => L=11.079 acc= 94% / t(ms):  56.7 312.4 313.3)
e349-i0055 => L=11.053 acc= 92% / t(ms):  58.0 317.1 318.7)
e349-i0057 => L=11.076 acc= 91% / t(ms):  59.7 317.8 323.7)
e349-i0059 => L=11.141 acc= 88% / t(ms):  59.1 318.6 326.0)
e349-i0061 => L=10.957 acc= 94% / t(ms):  59.1 324.7 325.8)
e349-i0063 => L=10.876 acc=100% / t(ms):  59.5 325.5 327.0)
e349-i0065 => L=11.792 acc= 88% / t(ms):  59.1 323.7 330.0)
e349-i0067 => L=10.965 acc= 96% / t(ms):  58.8 324.9 330.0)
e349-i0069 => L=11.139 acc= 88% / t(ms):  58.7 330.9 329.4)
e349-i0071 => L=11.506 acc= 94% / t(ms):  58.4 330.1 329.7)
e349-i0073 => L=11.104 acc= 88% / t(ms):  58.2 331.3 331.2)
e349-i0075 => L=11.106 acc= 89% / t(ms):  58.0 328.7 330.7)
e349-i0077 => L=11.117 acc= 90% / t(ms):

e351-i0058 => L=11.113 acc= 82% / t(ms):  52.9 285.5 302.7)
e351-i0060 => L=11.128 acc= 91% / t(ms):  53.4 295.6 310.3)
e351-i0062 => L=10.972 acc= 93% / t(ms):  54.2 301.3 317.6)
e351-i0064 => L=11.438 acc= 86% / t(ms):  55.2 311.5 318.9)
e351-i0066 => L=10.964 acc=100% / t(ms):  56.5 314.0 320.7)
e351-i0068 => L=11.052 acc= 89% / t(ms):  56.6 317.3 323.9)
e351-i0070 => L=11.042 acc= 94% / t(ms):  57.8 317.3 331.8)
e351-i0072 => L=11.839 acc= 94% / t(ms):  57.9 320.2 332.4)
e351-i0074 => L=10.987 acc= 95% / t(ms):  58.5 321.0 331.9)
e351-i0076 => L=11.093 acc= 96% / t(ms):  59.7 323.1 331.1)
e351-i0078 => L=11.051 acc=100% / t(ms):  60.0 323.1 330.6)
e351-i0080 => L=11.139 acc= 96% / t(ms):  60.5 324.9 332.2)
e351-i0082 => L=11.043 acc= 86% / t(ms):  60.0 322.6 332.3)
e351-i0084 => L=11.395 acc= 99% / t(ms):  59.5 322.3 334.7)
e351-i0086 => L=10.918 acc= 97% / t(ms):  59.6 323.9 335.0)
e351-i0088 => L=11.144 acc= 92% / t(ms):  59.1 325.7 334.6)
e351-i0090 => L=10.974 acc= 96% / t(ms):

e353-i0070 => L=11.088 acc= 97% / t(ms):  50.1 285.4 299.8)
e353-i0072 => L=11.067 acc=100% / t(ms):  53.5 296.5 308.2)
e353-i0074 => L=11.032 acc= 87% / t(ms):  54.3 297.7 311.3)
e353-i0076 => L=11.175 acc= 79% / t(ms):  56.0 314.1 320.4)
e353-i0078 => L=10.902 acc=100% / t(ms):  58.5 318.2 323.6)
e353-i0080 => L=10.933 acc=100% / t(ms):  56.3 314.2 320.2)
e353-i0082 => L=11.013 acc= 93% / t(ms):  55.7 306.8 316.7)
e353-i0084 => L=11.557 acc= 96% / t(ms):  54.1 304.7 318.6)
e353-i0086 => L=11.152 acc= 82% / t(ms):  54.5 305.7 321.1)
e353-i0088 => L=11.091 acc= 95% / t(ms):  53.9 303.4 315.8)
e353-i0090 => L=11.099 acc= 90% / t(ms):  53.4 298.8 323.5)
e353-i0092 => L=11.180 acc= 94% / t(ms):  52.9 299.9 322.7)
e353-i0094 => L=10.968 acc= 94% / t(ms):  53.8 302.9 322.4)
e353-i0096 => L=11.182 acc= 82% / t(ms):  53.4 298.2 318.4)
e353-i0098 => L=11.267 acc= 75% / t(ms):  52.4 297.1 316.8)
Validation : 10.0% (timings : 16.48 30.46)
Validation : 22.0% (timings : 28.46 52.09)
Validation : 3